**This Notebook shows what our hackathon product does to provide your schedule with weather data**

We need a function that retrieves weather forecast data from the Royal Dutch Meteorological Institute (KNMI)

In [1]:
import requests

def request_and_save_weather_data():

    params = (
        ('locatie', 'Amsterdam'),
        ('key', '85c9b987c6'),
    )

    response = requests.get('https://meteoserver.nl/api/uurverwachting_gfs.php', params=params)
    content = response.content

    file = open("weather.txt", "w+")
    file.write(str(response.content)[2:-1])
    print('wrote new weather data file')

lets call this function and see what we get back

In [10]:
    import json   

    with open("weather10.txt") as f2:
        data10 = json.load(f2)
        print(data10)

{'plaatsnaam': [{'plaats': 'Amsterdam'}], 'data': [{'tijd': '1568883600', 'tijd_nl': '19-09-2019 13:00', 'offset': '36', 'loc': '19569', 'temp': '15', 'winds': '1', 'windb': '1', 'windknp': '2', 'windkmh': '3.6', 'windr': '333', 'windrltr': 'NW', 'neersl': '0', 'luchtd': '1031', 'luchtdmmhg': '773.3', 'luchtdinhg': '30.45', 'rv': '26', 'gr': '182', 'hw': '74', 'mw': '0', 'lw': '94', 'tw': '0', 'cond': '5', 'ico': '7', 'samenv': 'Zonnig', 'icoon': 'zonnig'}, {'tijd': '1568887200', 'tijd_nl': '19-09-2019 14:00', 'offset': '37', 'loc': '19569', 'temp': '16', 'winds': '2', 'windb': '2', 'windknp': '4', 'windkmh': '7.2', 'windr': '1', 'windrltr': 'N', 'neersl': '0', 'luchtd': '1031', 'luchtdmmhg': '773.3', 'luchtdinhg': '30.45', 'rv': '26', 'gr': '245', 'hw': '62', 'mw': '0', 'lw': '85', 'tw': '0', 'cond': '5', 'ico': '7', 'samenv': 'Zonnig', 'icoon': 'zonnig'}, {'tijd': '1568890800', 'tijd_nl': '19-09-2019 15:00', 'offset': '38', 'loc': '19569', 'temp': '16', 'winds': '2', 'windb': '2', 'w

Next up, we'll define a function to extract and format the weather data for the next 10 days.\
we'll call this one later.

In [15]:
    def weather10(self):
        s3 = {}
        Temperature = []
        Date = []
        WindSpeed = []
        Precipitations = []
        for j in range(len(self.data10["data"])):
            Temperature.append(self.data10["data"][j]["temp"]), Date.append(self.data10["data"][j]["tijd_nl"])
            # To transform the date into daytime form use datetime.datetime.strptime(Date[x], "%d-%m-%Y %H:%M")
            WindSpeed.append(self.data10["data"][j]["windkmh"]), Precipitations.append(self.data10["data"][j]["neersl"])

        s3.update({"Date": Date})
        s3.update({"Temperature": Temperature})
        s3.update({"Wind Speed": WindSpeed})
        s3.update({"Precipitations": Precipitations})
        return s3

We need to define some constants that we are going to use to retrieve the schedule from the rooster.hva.nl mobile website

In [14]:
    from datetime import datetime, date
    
    class_group = ""
    API_PATH = "https://rooster.hva.nl/m/api/timetable?start="+str(date.today())+"T00%3A00%3A00.000%2B02%3A00&limit=10&key="
    API_DATA_FORMAT = "&format=json"
    GET_CLASS = "https://rooster.hva.nl/m/api/search/timetables?q="

In [39]:
    def get_date_time_name(date):
        weekDay = calendar.day_name[datetime.fromtimestamp((int(str(date)[:-3]))).weekday()]
        dayNumber = datetime.fromtimestamp(int(str(date)[:-3])).strftime("%d")
        monthName = datetime.fromtimestamp(int(str(date)[:-3])).strftime("%B")
        year = datetime.fromtimestamp(int(str(date)[:-3])).strftime("%Y")
        dateTimeName = (str(weekDay + " " + dayNumber + " " + monthName + " " + year).lower())

        return dateTimeName

Then we define a function that builds and sends a request to the schedule website using the parameter 'path' which represents a course code.

In [40]:
    def perform_request(path):

        # Set cookie to authorize the program to use the timetable API
        session = requests.session()
        session.get("https://rooster.hva.nl/m")
        payload = {
            'JSESSIONID': session.cookies.get_dict()
        }

        # Request student set by searching class
        get_class = session.get(GET_CLASS + path +
                                "&subscriptionGroup="+ str(datetime.now().year) +
                                "&type=studentset&limit=50" + API_DATA_FORMAT, params=payload)

        # Get student class
        student_set = ""
        if get_class.status_code == 200:
            class_json = json.loads(get_class.content)
            student_set = class_json['data'][0]['key']

        # Request timetable API for student set
        request = session.get(API_PATH + student_set + API_DATA_FORMAT, params=payload)
        if request.status_code == 200:
            return json.loads(request.content)
        return None

Then we need a function to combine the weahter data with the schedule data.

In [43]:
    def get_schedule_for_class_group(class_group):
        data = perform_request(class_group)
        if 'data' not in data:
            print("No 'results' in json")
            return None

        schedule = []
        for timetable in data['data']:

            # Get specific datetime for comparing with weather datetime
            timeTableStartDate = datetime.fromtimestamp(int(str(timetable['startDate'])[:-3])).strftime("%d-%m-%Y %H:00")
            timeTableEndDate = datetime.fromtimestamp(int(str(timetable['startDate'])[:-3])).strftime(
                "%d-%m-%Y %H:00")

            startTime = datetime.fromtimestamp(int(str(timetable['startDate'])[:-3])).strftime("%H:%M")
            endTime = datetime.fromtimestamp(int(str(timetable['endDate'])[:-3])).strftime("%H:%M")

            dateName = get_date_time_name(timetable['startDate'])
            weekNumber = (datetime.fromtimestamp(int(str(timetable['startDate'])[:-3])).strftime("%V"))

            schedule.append({
                'id': timetable['id'],
                'name': timetable['name'],
                'startTime': startTime,
                'endTime': endTime,
                'dateName': dateName,
                'weekNumber': weekNumber,
                'timeTableStartDate': timeTableStartDate,
                'timeTableEndDate': timeTableEndDate,
                'locations': timetable['locations']
            })

        return schedule

Let's call this function and see what happens

In [ ]:
    import calendar
    
    class_group = 'IVMSFS'

    schedule_result = (get_schedule_for_class_group(class_group))
    
    print(schedule_result)

This is the data that we use to resolve the html template and show the user a nice looking schedule with weather data.